In [1]:
# ! pip install --upgrade dune-client
# ! pip show dune-client

In [2]:
# Get L2 Revenue and post it to a database (csv in github for now)
# TODO - Integrate with BQ upload
import pandas as pd
import sys
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
sys.path.append("../helper_functions")
# import web3py_utils as w3py
# import duneapi_utils as du
import google_bq_utils as bqu
from web3 import Web3
from datetime import datetime, timezone
sys.path.pop()

import os

In [3]:
# https://github.com/ethereum-optimism/optimism/blob/b86522036ad11a91de1d1dadb6805167add83326/specs/predeploys.md?plain=1#L50

# Name, contract
fee_vaults = [
    ['SequencerFeeVault','0x4200000000000000000000000000000000000011'],
    ['BaseFeeVault','0x4200000000000000000000000000000000000019'],
    ['L1FeeVault','0x420000000000000000000000000000000000001A'],
]
bespoke_fee_vaults = [
        [255, 'Kroma: L1 Fee Vault', '0x4200000000000000000000000000000000000007'] #Kroma
]

# Aiming to eventually read from superchain-resitry + some non-superchain static adds
chains_rpcs = pd.read_csv('outputs/chain_metadata.csv', na_filter=False)
# print(chains_rpcs.columns)
chains_rpcs = chains_rpcs[~(chains_rpcs['rpc_url'] == '') & ~(chains_rpcs['op_based_version'].str.contains('legacy'))]
# chains_rpcs = chains_rpcs.values.tolist()
# print(chains_rpcs.sample(5))

# # Temp
# chains_rpcs = chains_rpcs[chains_rpcs['chain_name'] == 'kroma']
# chains_rpcs

In [4]:
# Calculate the method signature hash
method_signature = "totalProcessed()"
method_id = Web3.keccak(text=method_signature)[:4].hex()
# Verify the method ID
print(f"Method ID: {method_id}")

Method ID: 0x84411d65


In [5]:
df_columns = [
    'block_time', 'block_number', 'chain_name', 'vault_name', 
    'vault_address', 'alltime_revenue_native', 'chain_id'
    # 'gas_token', 'da_layer', 'is_superchain_registry' # Uncomment these if needed
]

In [6]:
data_arr = []

def process_chain(chain):
    chain_name = chain['chain_name']
    chain_id = chain['mainnet_chain_id']

    if chain_id:  # Check if chain_id is not empty
        chain_id = str(chain_id) 
    else:
        chain_id = None  # or keep it as an empty string or any default value you prefer

    print(chain_name + ' - ' + str(chain_id))
    rpc = chain['rpc_url']
    # gas_token = chain['gas_token']
    # da_layer = chain['da_layer']
    # is_superchain_registry = chain['superchain_registry']

    try:
        w3_conn = Web3(Web3.HTTPProvider(rpc))
        # Get the timestamp of the latest block
        block_timestamp = w3_conn.eth.get_block('latest').timestamp
        block_number = w3_conn.eth.get_block('latest').number
        # Convert the UNIX timestamp to a human-readable format
        block_datetime = datetime.fromtimestamp(block_timestamp, tz=timezone.utc)
        block_time = block_datetime.strftime('%Y-%m-%d %H:%M:%S')

        bespoke_vaults = [[item[1], item[2]] for item in bespoke_fee_vaults if str(item[0]) == str(chain_id)]
        iter_fee_vaults = fee_vaults + bespoke_vaults

        for vault in iter_fee_vaults:
            try:
                vault_name = vault[0]
                vault_address = vault[1]
                # Call the function directly using eth_call
                response = w3_conn.eth.call({
                    'to': vault_address,
                    'data': method_id
                })
                wei_balance = w3_conn.eth.get_balance(vault_address)
                # Decode the result (assuming the function returns a uint256)
                proxy_processed_wei = Web3.to_int(hexstr=response.hex())
                
                alltime_revenue_wei = proxy_processed_wei + wei_balance
                alltime_revenue_native = alltime_revenue_wei / 1e18
                # print(alltime_revenue_native)
                tmp = pd.DataFrame(
                        [[block_time, block_number, chain_name, vault_name, vault_address, alltime_revenue_native, chain_id]]
                        , columns=df_columns
                        )
                data_arr.append(tmp)
                time.sleep(1)
            except Exception as e:
                print(f"Error processing vault {vault_name} on chain {chain_name}: {e}")
                continue  # Skip to the next vault
    except Exception as e:
        print(f"Error processing chain {chain_name}: {e}")

# Use ThreadPoolExecutor to parallelize the API calls
with ThreadPoolExecutor(max_workers=99) as executor: # Max Workers can be big, since we only do 3 calls per RPC
    futures = [executor.submit(process_chain, chain) for index, chain in chains_rpcs.iterrows()]
    for future in as_completed(futures):
        future.result()  # This will raise any exceptions caught during the execution


op - 10
zora - 7777777
base - 8453
mode - 34443
pontem - 8866
lisk - 1135
metal - 1750
mint - 185
xterio - 2702128
kroma - 255
orderly - 291
fraxtal - 252
redstone - 690
cyber - 7560
ham - 5112
lyra - 957
bob - 60808
hypr - 881
loot chain - 5151706
rollux - 570
pgn - 424
manta_pacific - 169
opbnb - 204
syndicate_frame - 5101
ancient8 - 888888888
goldchain - 4653
karak - 2410
blast - 81457
stack - 78225
rss3 - 12553
lambda - 56026
swan - 254
ethxy - 969
superseed - 5330
zircuit - 48900
polynomial - 8008
race - 6805
funki - 33979
optopia - 62050
donatuz - 42026
matchain - 698
Error processing chain loot chain: The field extraData is 97 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#proof-of-authority for more details. The full extraData is: HexBytes('0xd98301090a846765746889676f312e31352e3133856c696e75780000000000003f01d915d9a5bdd0023dcf8b6080c9bfbb082e1b5ee5ed36f712b2e2486551c07fc542b4262

In [7]:

# for index, chain in chains_rpcs.iterrows():
#     chain_name = chain['chain_name']
#     chain_id = chain['mainnet_chain_id']

#     # if chain_id:  # Check if chain_id is not empty
#     #     chain_id = int(float(chain_id))  # Convert to float first, then to int
#     # else:
#     #     chain_id = None  # or keep it as an empty string or any default value you prefer
        
#     print(chain_name + ' - ' + str(chain_id))
#     rpc = chain['rpc_url']
#     # gas_token = chain['gas_token']
#     # da_layer = chain['da_layer']
#     # is_superchain_registry = chain['superchain_registry']

#     try:
#         w3_conn = Web3(Web3.HTTPProvider(rpc))
#         # Get the timestamp of the latest block
#         block_timestamp = w3_conn.eth.get_block('latest').timestamp
#         block_number = w3_conn.eth.get_block('latest').number
#         # Convert the UNIX timestamp to a human-readable format
#         block_datetime = datetime.fromtimestamp(block_timestamp, tz=timezone.utc)
#         block_time = block_datetime.strftime('%Y-%m-%d %H:%M:%S')

#         for vault in fee_vaults:
#             vault_name = vault[0]
#             vault_address = vault[1]
#             # Call the function directly using eth_call
#             response = w3_conn.eth.call({
#                 'to': vault_address,
#                 'data': method_id
#             })
#             wei_balance = w3_conn.eth.get_balance(vault_address)
#             # Decode the result (assuming the function returns a uint256)
#             proxy_processed_wei = Web3.to_int(hexstr=response.hex())
            
#             alltime_revenue_wei = proxy_processed_wei+wei_balance
#             alltime_revenue_native = alltime_revenue_wei/1e18

#             print(chain_name + ' | ' + vault_name + ': ' \
#                 + str(proxy_processed_wei) + ' | bal: ' + str(wei_balance)\
#                 + ' | total eth: ' + str( (alltime_revenue_native) )
#                 )
            
#             tmp = pd.DataFrame(
#                     [[block_time, block_number, chain_name, vault_name, vault_address, alltime_revenue_native, chain_id]]#, gas_token, da_layer, is_superchain_registry]]
#                     ,columns =df_columns
#                     )
#             data_arr.append(tmp)
#             time.sleep(1)
#     except:
#         continue

In [8]:
data_df = pd.concat(data_arr)
data_df['block_time'] = pd.to_datetime(data_df['block_time'])

In [9]:
file_path = 'outputs/all_time_revenue_data.csv'

# data_df.sample()

In [10]:
# Check if the file exists
if os.path.exists(file_path):
    # Read the existing CSV file
    existing_df = pd.read_csv(file_path)
    # Ensure the existing file has the same columns
    existing_df = existing_df.reindex(columns=df_columns)
    data_df = data_df.reindex(columns=df_columns)
    # Append without writing the header
    data_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    # If file doesn't exist, create it and write the header
    data_df.to_csv(file_path, mode='w', header=True, index=False)

In [12]:
# # Overwrite to Dune Table
# dune_df = pd.read_csv(file_path)
# print(dune_df.sample(5))
# du.write_dune_api_from_pandas(dune_df, 'op_stack_chains_cumulative_revenue_snapshots',\
#                              'Snapshots of All-Time (cumulative) revenue for fee vaults on OP Stack Chains. Pulled from RPCs - metadata in op_stack_chains_chain_rpc_metdata')

In [13]:
# #Insert Updates to Dune Table
# create_namespace = 'oplabspbc'
# create_table_name = 'op_stack_chains_cumulative_revenue_snapshots'
# create_table_description = 'Snapshots of All-Time (cumulative) revenue for fee vaults on OP Stack Chains. Pulled from RPCs - metadata in op_stack_chains_chain_rpc_metdata'
# # try:
# # du.create_dune_table(data_df, namespace = create_namespace
# #                         , table_name = create_table_name
# #                         , table_description = create_table_description)

# # except:
#         # print('error creating')
# du.insert_dune_api_from_pandas(data_df, namespace = create_namespace,table_name = create_table_name)

# du.write_dune_api_from_pandas(chains_rpcs, 'op_stack_chains_chain_rpc_metdata',\
#                              'Chain metadata - used to join with op_stack_chains_cumulative_revenue_snapshots')

In [14]:
bq_cols = ['block_time','block_number','chain_name','vault_name','vault_address','alltime_revenue_native','chain_id']

In [15]:
# dune_df.sample(5)

In [16]:
data_df['chain_id'] = data_df['chain_id'].astype('string')
data_df['block_time'] = pd.to_datetime(data_df['block_time'])
# dune_df['block_time'] = pd.to_datetime(dune_df['block_time'])
data_df.dtypes

block_time                datetime64[ns]
block_number                       int64
chain_name                        object
vault_name                        object
vault_address                     object
alltime_revenue_native           float64
chain_id                  string[python]
dtype: object

In [17]:
data_df

,block_time,block_number,chain_name,vault_name,vault_address,alltime_revenue_native,chain_id
0,2024-08-29 19:56:37,19086025,base,SequencerFeeVault,0x4200000000000000000000000000000000000011,12333.787483,8453
0,2024-08-29 19:56:37,8075791,blast,SequencerFeeVault,0x4200000000000000000000000000000000000011,0.010000,81457
0,2024-08-29 19:56:37,33207674,opbnb,SequencerFeeVault,0x4200000000000000000000000000000000000011,1259.525800,204
0,2024-08-29 19:56:37,19133779,zora,SequencerFeeVault,0x4200000000000000000000000000000000000011,542.132865,7777777
0,2024-08-29 19:56:37,9075343,fraxtal,SequencerFeeVault,0x4200000000000000000000000000000000000011,14.532960,252
...,...,...,...,...,...,...,...
0,2024-08-29 19:56:39,3652611,matchain,L1FeeVault,0x420000000000000000000000000000000000001A,0.027338,698
0,2024-08-29 19:56:37,2512591,zircuit,L1FeeVault,0x420000000000000000000000000000000000001A,0.008445,48900
0,2024-08-29 19:56:39,5807888,lambda,L1FeeVault,0x420000000000000000000000000000000000001A,0.063331,56026
0,2024-08-29 19:56:39,4813712,optopia,L1FeeVault,0x420000000000000000000000000000000000001A,9.420181,62050


In [18]:
dataset_name = 'rpc_table_uploads'
table_name = 'hourly_cumulative_l2_revenue_snapshots'

In [19]:


# Write All to BQ Table
# bqu.write_df_to_bq_table(df = dune_df[bq_cols], table_id = table_name, dataset_id = dataset_name)

# Write Updates to BQ Table
unique_cols = ['block_time', 'chain_name', 'chain_id', 'vault_name']
bqu.write_df_to_bq_table(df = data_df[bq_cols], table_id = table_name, dataset_id = dataset_name, write_mode='append')

Start Writing rpc_table_uploads.hourly_cumulative_l2_revenue_snapshots
Data loaded successfully to rpc_table_uploads.hourly_cumulative_l2_revenue_snapshots


In [20]:
# # Backfills
# # Read the CSV file into a DataFrame
# df = pd.read_csv(file_path)

# # Filter the DataFrame where chain_name is 'kroma'
# filtered_df = df[df['chain_name'] == 'kroma']
# filtered_df['chain_id'] = filtered_df['chain_id'].astype('string')
# filtered_df['block_time'] = pd.to_datetime(filtered_df['block_time'])

# bqu.append_and_upsert_df_to_bq_table(df = filtered_df[bq_cols], table_id = table_name, dataset_id = dataset_name, unique_keys=['block_time', 'chain_name', 'vault_address'])